In [1]:
import datetime as dt
import pandas as pd

from de.processor.dart import DART_FUNDAMENTAL_PROCESSOR
from de.processor.fdr import FDR_INFO_PROCESSOR, FDR_OHLCV_PREPROCESSOR

from de.processor.factor.fundamental import FUNDAMENTAL_FACTOR_PROCESSOR
from de.processor.factor.technical import LINEAR_COEFFICENT_FACTOR_PROCESSOR, MOVING_AVERAGE_FACTOR_PROCESSOR


from de.analyser.factor import FACTOR_ANALYSER
from de.model_paramter import MODEL_PARAMTER_PROCESSOR

In [2]:
# CFG

In [3]:
CFG = {
    "FUNDAMENTAL": {
        "PROCESSOR": {
            "ohlcv_recent_n": 10,
        },
    },
    "LINEAR_COEF": {
        "PROCESSOR": {
            "factors": ["Close", "VolumeRotation"],
            "moving_average_window": 10,
            "coef_recent_n": 20,
        }
    },
    "MOVING_AVERAGE": {
        "PROCESSOR": {
            "factors": ["Close", "VolumeRotation"],
            "short_term_window": round((7 * 2) * (5 / 7)),
            "long_term_window": round((7 * 8) * (5 / 7)),
            "signal_recent_n": 20,
        }
    },
    "FACTOR_ANALYSIS": {
        "profit_recent_n": 20,
    },
}
recen_n = CFG["FACTOR_ANALYSIS"]["profit_recent_n"]
analysis_date = (dt.date.today() - dt.timedelta(days=recen_n)).strftime("%Y-%m-%d")
CFG["analysis_date"] = analysis_date

In [4]:
# Load

In [5]:
# dart_fundamental
dart_fundamental_df = pd.read_csv("./de/data/dart_fundamental_df.csv", index_col=0)
dart_fundamental_processor = DART_FUNDAMENTAL_PROCESSOR(dart_fundamental_df)

In [6]:
# fdr_info
fdr_info_df = pd.read_csv("./de/data/fdr_info_df.csv", index_col=0)
fdr_info_processor = FDR_INFO_PROCESSOR(fdr_info_df)

In [7]:
# fdr_ohlcv
fdr_ohlcv_df = pd.read_csv("./de/data/fdr_ohlcv_df.csv", index_col=0)
fdr_ohlcv_preprocessor = FDR_OHLCV_PREPROCESSOR(fdr_ohlcv_df)
pps_fdr_ohlcv_df = fdr_ohlcv_preprocessor.get_pps_fdr_ohlcv_df()

In [8]:
# split ohlcv
future_ohlcv_df = pps_fdr_ohlcv_df[CFG["analysis_date"] < pps_fdr_ohlcv_df["Date"]].copy()
past_ohlcv_df = pps_fdr_ohlcv_df[pps_fdr_ohlcv_df["Date"] < CFG["analysis_date"]].copy()

In [9]:
# Parameter
model_paramter_processor = MODEL_PARAMTER_PROCESSOR(CFG["FACTOR_ANALYSIS"])

In [10]:
# Fundamental

# fundamental_factors_df
fundamental_factor_processor = FUNDAMENTAL_FACTOR_PROCESSOR(
    past_ohlcv_df, dart_fundamental_processor, fdr_info_processor
)
fundamental_factors_df = fundamental_factor_processor.get_fundamental_factor_df(CFG["FUNDAMENTAL"]["PROCESSOR"])

# analyser
fundamental_factor_analyser = FACTOR_ANALYSER(fundamental_factors_df)

fundamental_params_dict = model_paramter_processor.get_params_dict(future_ohlcv_df, fundamental_factor_analyser)
fundamental_params_df = pd.DataFrame().from_dict(fundamental_params_dict, orient="index", columns=["Value", "Variance"])
fundamental_params_df.nlargest(10, "Value").nsmallest(5, "Variance")

,Value,Variance
"((CEPP, 0.0, 0.3), (TLR, 0.4, 0.7))",0.052469,0.001084
"((CEPP, 0.1, 0.4), (TLR, 0.4, 0.7))",0.053669,0.001136
"((CEPP, 0.0, 0.3), (TLR, 0.2, 0.5))",0.055498,0.001208
"((CEPP, 0.1, 0.4), (TLR, 0.3, 0.6))",0.057192,0.001349
"((NPP, 0.3, 0.6), (TLR, 0.5, 0.8))",0.050593,0.001807


In [11]:
# Technical

# linear_coef_factors_df
linear_coefficent_factor_processor = LINEAR_COEFFICENT_FACTOR_PROCESSOR(past_ohlcv_df, fdr_info_processor)
linear_coef_factors_df = linear_coefficent_factor_processor.get_linear_coef_factor_df(CFG["LINEAR_COEF"]["PROCESSOR"])

# Analysis
linear_coef_factor_analyser = FACTOR_ANALYSER(linear_coef_factors_df)

linear_coef_params_dict = model_paramter_processor.get_params_dict(future_ohlcv_df, linear_coef_factor_analyser)
linear_coef_params_df = pd.DataFrame().from_dict(linear_coef_params_dict, orient="index", columns=["Value", "Variance"])
linear_coef_params_df.nlargest(10, "Value").nsmallest(5, "Variance")

,Value,Variance
"((VolumeRotation, 0.4, 0.7), (Close, 0.2, 0.5))",0.042757,0.000857
"((VolumeRotation, 0.3, 0.6), (Close, 0.1, 0.4))",0.043575,0.001023
"((VolumeRotation, 0.3, 0.6), (Close, 0.2, 0.5))",0.045764,0.001139
"((VolumeRotation, 0.5, 0.8), (Close, 0.2, 0.5))",0.046132,0.001145
"((VolumeRotation, 0.4, 0.7), (Close, 0.1, 0.4))",0.047182,0.001183


In [12]:
# Technical

# moving_average_factors_df
moving_average_factor_processor = MOVING_AVERAGE_FACTOR_PROCESSOR(past_ohlcv_df, fdr_info_processor)
moving_average_factors_df = moving_average_factor_processor.get_moving_average_factor_df(
    CFG["MOVING_AVERAGE"]["PROCESSOR"]
)

# Analysis
moving_average_factor_analyser = FACTOR_ANALYSER(moving_average_factors_df)

moving_average_params_dict = model_paramter_processor.get_params_dict(future_ohlcv_df, moving_average_factor_analyser)
moving_average_params_df = pd.DataFrame().from_dict(moving_average_params_dict, orient="index", columns=["Value", "Variance"])
moving_average_params_df.nlargest(10, "Value").nsmallest(5, "Variance")

,Value,Variance
"((VolumeRotation, 0.5, 0.8), (Close, 0.2, 0.5))",0.042939,0.001163
"((VolumeRotation, 0.7, 1.0), (Close, 0.5, 0.8))",0.046618,0.001189
"((VolumeRotation, 0.4, 0.7), (Close, 0.3, 0.6))",0.048593,0.001346
"((VolumeRotation, 0.3, 0.6), (Close, 0.3, 0.6))",0.050039,0.001516
"((VolumeRotation, 0.5, 0.8), (Close, 0.3, 0.6))",0.050133,0.001813
